<a href="https://colab.research.google.com/github/senaruna0112/yahoonews/blob/main/yahoonews_NCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# webスクレイピング

## Colab上でWeb Driverを入れてスクレイピングできる環境を作る

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 958 kB 4.9 MB/s 
     |████████████████████████████████| 138 kB 50.6 MB/s 
     |████████████████████████████████| 359 kB 48.4 MB/s 
     |████████████████████████████████| 3.6 MB 42.6 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

# yahooニュースのスクレイピング

In [157]:
# import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd

# initialize chrome driver (settings)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# create webdriver
driver = webdriver.Chrome('chromedriver',options=options)

In [158]:
MAX_PAGE = 4
href_list = []

for i in range(1, MAX_PAGE+1):
  print(f"https://news.yahoo.co.jp/topics/business?page={i}")
  driver.get(f"https://news.yahoo.co.jp/topics/business?page={i}")
  # html = driver.page_source.encode('utf-8')
  html = driver.page_source

  # bs4の設定
  soup = BeautifulSoup(html, 'html.parser')
#   print(soup)

  # 現在いるページにある記事のリンク一覧
  tmp = soup.find_all(class_="newsFeed_item_link")
  print(tmp)
  # linkのtextをとってくる
  for tar in tmp:
    href_list.append(tar.get("href"))
  time.sleep(1)

https://news.yahoo.co.jp/topics/business?page=1
[<a class="newsFeed_item_link" data-cl-params="_cl_vmodule:st_topics;_cl_link:title;_cl_position:1;" data-rapid_p="46" data-ylk="rsec:st_topics;slk:title;pos:1;" href="https://news.yahoo.co.jp/pickup/6419068"><div class="newsFeed_item_thumbnail"><div class="sc-dVhcbM jKVCqr thumbnail thumbnail-small"><img alt="" class="sc-eqIVtm koAuUY" loading="lazy" src="https://news-pctr.c.yimg.jp/t/news-topics/images/tpc/2022/2/23/8138674cc922f8c5f64089196ac0a855e1c0541b103243a5f03cfb24ad8d560c.jpg"/></div></div><div class="newsFeed_item_text"><div class="newsFeed_item_title">ワークマンがキャンプ参入 なぜ今</div><div class="newsFeed_item_sub"><div class="newsFeed_item_sourceWrap"><time class="newsFeed_item_date">2/23(水) 10:10</time></div></div></div></a>, <a class="newsFeed_item_link" data-cl-params="_cl_vmodule:st_topics;_cl_link:title;_cl_position:2;" data-rapid_p="47" data-ylk="rsec:st_topics;slk:title;pos:2;" href="https://news.yahoo.co.jp/pickup/6419066"><div c

In [159]:
titles, contents = [], []
for target_url in href_list:
  driver.get(target_url)
  title = driver.title
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  url_all = soup.find_all(class_="sc-lkYwMk cIUcbi")[0].find_all(class_="sc-iLWYPX gTRgYH")[0].get("href")
  print(url_all)
#   url_all = soup.find(class_="sc-bQEgQf dXgvwT").get("href")
#   url_all = soup.find_all(class_="sc-bQEgQf dXgvwT")[0].find_all(class_="sc-iScGEx PmDTp")[0].get("href") #井上さんが書いた。　classが変更になっている？
  # 全文
  driver.get(url_all)  # 全文があるリンクにアクセス
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  title = soup.find('title').text  # タイトル取得
  text = soup.find(class_="article_body highLightSearchTarget")  # 本文を取得
  if text is None: 
    print(f"取得できませんでした: {target_url}")
    continue
  text = text.get_text()
  print(title)
  titles.append(title)
  contents.append(text)
  time.sleep(1)

https://news.yahoo.co.jp/articles/ee708f6a4a155d4001280e3c675c26e17097ab3a
「ワークマン」がキャンプ市場に本格参入　「周回遅れ」で繰り出す販売戦略（SankeiBiz） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/3aba2626fb91ff25ad210b961e33bbcd7f14b185
ミツカン「味ぽん」誕生のきっかけとなった博多名物は？　なぜ全国制覇し食卓の定番に（NIKKEI STYLE） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/f1281b45e85dfbcb031e355dabded5c41898015b
NY株続落、482ドル安　ウクライナ本格侵攻を警戒（共同通信） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/f09837bf58ad3071e6795a9c916c6a13843ebe3d
ウクライナ緊迫、株大幅続落　金最高値、リスク回避強まる　原油も急伸（時事通信） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/186196c4285f089a41dc8cfb5901256d03f988d5
テレ朝　早河洋社長・COO　前社長の辞任騒動を陳謝「お詫び申し上げる」経費私的使用等で10日付辞任（スポニチアネックス） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/a089d3d31efcdb05c07fc04bb1c049f524d4ac85
霞が関の残業代、初の400億円台に　異例の18%大幅増　22年度（朝日新聞デジタル） - Yahoo!ニュース
https://news.yahoo.co.jp/byline/nakamuratomohiko/20220222-00283203
取得できませんでした: https://news.yahoo.co.jp/pickup/6419014
https://news.yahoo.co.jp/articles/8c6a855cabacf765b177f9171cb8d3afd7cfbf69


In [160]:
df = pd.DataFrame([[title, content] for title, content in zip(titles, contents)], columns=["title", "text"])
df.to_csv("yahoo_news_econ.csv",encoding='utf_8_sig')

# 実装してみる

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("/content/yahoo_news_econ2.csv", sep=",")

In [8]:
df.drop(df.columns[0], axis=1)

,title,text
0,なぜタイヤ脱落『9割以上が左後輪』なのか…専門家が指摘する“日本の道路事情”と“ナットの締め...,国土交通省が公開している実験の映像 岐阜県中津川市の中央道で1月、走行中のダンプカーからタイ...
1,新しい客、獲得できるチャンスなのに… 「取材拒否」の飲食店がある理由（オトナンサー） - Y...,なぜ取材拒否？ 飲食店の中には、テレビや雑誌の取材を一切受けない、いわゆる「取材拒否の店」が...
2,【速報】「まん延防止措置」追加適用で“夜の人出”減る 札幌36.5％ 大阪18.6％減（FN...,FNNプライムオンライン27日から追加で「まん延防止等重点措置」が適用された地域では、28日...
3,飲み会後のナイト需要消失が打撃 カラオケ市場、コロナ前から6割減へ 「歌わないカラオケ」で回...,「まん延防止」などで再び遠のく客足 当面は苦しい状況続くまん延防止などで居酒屋客など「ナイト...
4,生乳廃棄問題、給食ない春休みに再燃の可能性も 業界団体が見通し（朝日新聞デジタル） - Ya...,牛乳の消費拡大をPRする金子原二郎農林水産相（中央）ら=2021年12月17日、東京都千代田...
...,...,...
80,しょうゆやジャム…値上げ春先以降も 家計への影響懸念（産経新聞） - Yahoo!ニュース,消費者物価指数が上昇。値上げされる商品など。調理油類も値上げに＝２１日、大阪市城東区のスーパ...
81,ガソリンスタンドの屋根はなぜ平らなのか？ 雪が降ったらどうしてる？（くるまのニュース） - ...,「平ら」は合理性ゆえ ガソリンスタンドを覆う屋根（上屋）の多くは、水平に造られています。なぜ...
82,歌会はなぜ楽しい アイドルやホストも参入、コロナ禍の短歌ブーム（産経新聞） - Yahoo!...,女性アイドル４人らが出演した「アイドル歌会＠詠い納め」。「無記名でも誰の短歌か分かる」と好評...
83,「まさかマイナス？」8歳の貯金箱 駆け込みでATM故障も（時事通信） - Yahoo!ニュース,大量の小銭の行方は… ゆうちょ銀行が2022年1月17日に「硬貨取扱料金」を新設し、硬貨を預...


In [9]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [10]:
import MeCab
mecab = MeCab.Tagger()
import re
from collections import Counter

##テキストをリスト化する

In [11]:
def dftotext(df):
    texts=[]
    for i in range(len(df)):
        texts.append(df["text"][i])
    return(texts)

In [12]:
texts=dftotext(df)


In [13]:
#似たようなニュースを摘出する
num=[11,15,16,17,26,28,29,38,46,54,59,63,64]
texts2 =[]
for i in range(len(df)):
    if i in num:
        texts2.append(df["text"][i])

テキストの単語ごとの多次元リストの作成する関数

In [195]:
def get_keywords_list(texts):
    keywords = [[] for i in range(len(texts))]
    for i in range(len(texts)):
        tokenizer = MeCab.Tagger("-Ochasen")
        tokenizer.parse("")
        node = tokenizer.parseToNode(texts[i])
        while node:
            if node.feature.split(",")[0] == u"名詞":
                keywords[i].append(node.surface)
            elif node.feature.split(",")[0] == u"形容詞":
                keywords[i].append(node.feature.split(",")[6])
            elif node.feature.split(",")[0] == u"動詞":
                keywords[i].append(node.feature.split(",")[6])
            node = node.next
    return(keywords)

In [ ]:
keywords=get_keywords_list(texts)
keywords

## ワードのカウント

In [17]:
from collections import defaultdict

In [18]:
def word_freqs(keywords):
    word_freqs =[]
    for i in range(len(keywords)):
        word_freqs.append(defaultdict(int))
        for keyword in keywords[i]:
            word_freqs[i][keyword] += 1
    return(word_freqs)

In [ ]:
word_freqs = word_freqs(keywords)
word_freqs

In [20]:
from sklearn.feature_extraction import DictVectorizer
V = DictVectorizer()
counts = [[] for i in range(len(texts))]
for i in range(len(texts)):
    counts[i] = V.fit_transform(word_freqs[i]).toarray() 

In [ ]:
counts # 次元数が全然違うので前処理がない

# entitiesの次元を揃えるアプローチ

In [22]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

## bag of wods のdocument-entity Matrix A1 を作成

In [23]:
koyu = []
def split_text_only_nva(text):
    tagger = MeCab.Tagger()

    words = []
    for c in tagger.parse(text).splitlines()[:-1]:
        surface, feature = c.split('\t')
        pos = feature.split(',')[0]
        pos2 = feature.split(',')[1]

        if pos2 == "固有名詞":
            koyu.append(surface)

        if pos == '名詞':
            words.append(surface)
        if pos =="動詞":
            words.append(surface)
        if pos =="形容詞":
            words.append(surface)
    return ' '.join(words)


In [100]:
def get_bow_matrixA(texts):
    messages_list = []
    for i in range(len(texts)):
        messages_list.append(split_text_only_nva(texts[i]))
    messages_list
    docs = np.array(messages_list)
    count = CountVectorizer()
    bags = count.fit_transform(docs)
    matrix=bags.toarray()
    return(matrix)

In [107]:
A1 = get_bow_matrixA(texts)
A1.shape

(85, 4285)

In [146]:
#アプローチ①：取り出したdocumentsのentitiesの次元でmatrixを作成
A12= get_bow_matrixA(texts2)
A12.shape

(13, 567)

In [108]:
#アプローチ⓶　entitiesの次元数は元のmatrixAに合わせて、documentの次元を変更してみる
num=[11,15,16,17,26,28,29,38,46,54,59,63,64]
omit_num=[] 
for i in range(len(texts)):
    if i not in num:
        omit_num.append(i)
A13 = np.delete(A1, omit_num, 0)
A13.shape

(13, 4285)

In [ ]:
set(koyu) # 固有名詞のリスト

##tf-idf のdocument-entity Matrix A2 を作成

In [95]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# tf-idf
def get_tf_idf_matrixA(texts):
    messages_list = []
    for i in range(len(texts)):
        messages_list.append(split_text_only_nva(texts[i]))
    messages_list
    docs = np.array(messages_list)
    count = CountVectorizer()
    bags = count.fit_transform(docs)
    tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True) #norm='l2'を指定すると単語ベクトルの長さが1になるようコサイン正規化
    np.set_printoptions(precision=2)
    tf_idf = tfidf.fit_transform(bags)
    matrix=tf_idf.toarray()
    return(matrix)

In [87]:
A2=get_tf_idf_matrixA(texts)

(85, 4285)

In [88]:
#アプローチ①　取り出したdocumentsのentitiesの次元でmatrixを作成
A22=get_tf_idf_matrixA(texts2)

In [96]:
#アプローチ⓶
for i in range(len(texts)):
    if i not in num:
        omit_num.append(i)
A23 = np.delete(A2, omit_num, 0)
A23.shape

(13, 4285)

## NCIを計算する 


In [98]:
from scipy.linalg import svd 
import math
#引数はmatrixAとドキュメントの数
def compute_NCI(x,num_doc):
    U, S, V = svd(x) 
    S = sorted(S, reverse=True)
    print(S)
    k= 5
    sigma = S[:k]
    total =0
    for i in range(len(sigma)):
        a= sigma[i]
        sig2 = a*a
        totalall = total +sig2
        total = totalall
    total
    NCI= math.sqrt(total)
    print(NCI/num_doc)

In [103]:
#全ての文書
compute_NCI(A1,85)

[90.66764142549242, 55.776814217329125, 50.55189055994572, 48.05822021087853, 44.0729572019731, 39.92693944286653, 37.8918024003863, 35.40609562560275, 33.6696212065572, 32.63780792838363, 30.616413068738353, 28.518522487429987, 26.777402902793746, 25.94566702252166, 25.73432490008326, 25.302689005694486, 25.086615031840584, 23.99643273264958, 23.732263851181184, 23.024143214974952, 22.50991889136065, 22.23791700294391, 22.00984577619387, 21.617452328901194, 20.953232206058754, 20.583632250549226, 20.31272026215464, 19.86214841443408, 19.610953339236822, 19.492703383617393, 19.26214197334499, 18.785719352700827, 18.532119426888976, 18.275246717797604, 17.90330062947156, 17.56931731590487, 17.357656151327763, 16.547143617736804, 16.20520922228791, 15.752584215093018, 15.565155712721369, 15.404699204583933, 15.096795341201817, 14.93794781607597, 14.839923419231694, 14.746521130526284, 14.373414907051432, 14.103505467441213, 13.788088967199382, 13.556749185661321, 13.464417447220466, 13.3

In [105]:
# アプローチ①類似度が高いニュースを集めて計算（株系のニュース）
compute_NCI(A12,14)# NCIが高い値で出てることから成功していることが分かる 

[34.36259527256175, 26.364379041138285, 17.587753633836098, 15.617406646099333, 12.95757613655653, 11.412909078420041, 10.17219269753763, 9.306308236650862, 7.777159306360998, 6.607418622806898, 5.919499087147388, 4.413874517988929, 8.969744731841421e-15]
3.6400547229427525


In [109]:
#アプローチ2 次元を揃える
compute_NCI(A13,14)
##同じ結果なのがおかしい

[34.36259527256176, 26.36437904113831, 17.587753633836112, 15.617406646099331, 12.957576136556526, 11.412909078420034, 10.172192697537639, 9.306308236650873, 7.777159306361002, 6.6074186228068985, 5.919499087147388, 4.413874517988929, 1.1848016885578758e-15]
3.6400547229427542


TF-IDF のNCI

In [117]:
#全ての文書
compute_NCI(A2,85)

[2.325560112284184, 1.8711691287972068, 1.6034650130119048, 1.3922143886997458, 1.3529261892915574, 1.3347778180622314, 1.3090570508957955, 1.2692634920086787, 1.228212576365401, 1.2009201562079115, 1.1941546692574174, 1.1315218830133045, 1.1133066021373779, 1.1055643477110928, 1.0919953959818, 1.0824635373055362, 1.0701251234263471, 1.0667914256170148, 1.0537797985407937, 1.044145142915895, 1.0364145662409967, 1.0300474476029333, 1.0298342480777574, 1.0244670154498607, 1.0168565705569723, 1.0128561229437143, 1.0103389446202715, 1.0064545650925885, 1.0034195953087894, 0.9991154259083871, 0.9964108188674311, 0.99473116469086, 0.9875958066149035, 0.9858926802904362, 0.98187158071754, 0.9746008843827912, 0.9709564494768443, 0.9667368062395028, 0.9658074442384336, 0.964258158536548, 0.9609135127804673, 0.9577933186054042, 0.9542865784665305, 0.9497690366271266, 0.9441901934821278, 0.9422292538990464, 0.9375068693080975, 0.9340595119966475, 0.9302913177957524, 0.9244176954928096, 0.92150043

In [131]:
#アプローチ①　次元をセレクトニュースに合わせる
compute_NCI(A22,14)


[1.7400736767180773, 1.3383059350028859, 1.0694670711702623, 1.0173660056038827, 0.9917465315590659, 0.9466800857744168, 0.9230516096700981, 0.8894325787951433, 0.8315425420276895, 0.8261252858455989, 0.7582195991828222, 0.7283992837411648, 1.5517509388774433e-15]
0.20179391594718168


In [115]:
#アプローチ2 次元をすべての文書に揃える
compute_NCI(A23,14)

[1.8979836935138126, 1.3429918525479514, 1.0343739403148504, 1.0192498751439518, 0.9822931341857642, 0.9315210069932829, 0.8879374373103104, 0.8444791002956782, 0.7955997787547264, 0.7820038380803916, 0.6885047382926001, 0.6576277138563938, 3.5816589332786254e-16]
0.2079990773957467


##論文と同じoccurrenceのmatrixを作成する

In [118]:
#paper と同じoccurrence のmatrix
def get_ooccurence_MatrixA(matrix):
    for i in range(len(matrix)):
        for j in range(matrix.shape[1]) :
            if matrix[i][j] >= 1:
                matrix[i][j] = 1
    return(matrix)

In [134]:
A1_occurence= get_ooccurence_MatrixA(A1)
A1_occurence.shape
compute_NCI(A1_occurence,85)

[32.81055026737786, 22.49892924628584, 17.728924302493873, 16.66231043393337, 16.095431193260943, 15.736164999168498, 15.153009818097232, 14.149603780555527, 13.76537348533419, 13.435559733098042, 13.181913219627658, 12.940805946838509, 12.65848138331492, 12.557757406189863, 12.432302142177786, 12.325145943118086, 12.180286665122614, 12.080831828415882, 12.005100049267233, 11.83053235060529, 11.795006919670021, 11.714906060408904, 11.501538240942594, 11.465317815761349, 11.330447611716549, 11.20848787749875, 11.151624785874056, 10.993957285134139, 10.843153015953787, 10.614323534761763, 10.596162944698762, 10.516230649018084, 10.46617615766244, 10.388827326797562, 10.312287829866833, 10.15665044651401, 10.068222298163553, 9.858370666324346, 9.79756218282144, 9.571214431053951, 9.530531342934976, 9.494938961321745, 9.429947365436805, 9.33262046568132, 9.14916273638405, 9.07335525028681, 9.05072100907842, 8.827823132202457, 8.622747166707486, 8.60504059275964, 8.57290143560173, 8.5086879

In [148]:
A12occurrence=get_ooccurence_MatrixA(A12)
A12occurrence.shape
compute_NCI(A12occurrence,14)

[16.70149815252203, 11.684072248427412, 11.00114925166548, 10.57217197264465, 9.587247176871234, 7.83007200470622, 7.559729960931652, 6.945628950533064, 6.207138033130197, 5.705895734189238, 5.320699275302102, 4.090726827626974, 1.5655997692440233e-14]
1.9432890779738312


In [149]:
A13occurrence=get_ooccurence_MatrixA(A13)
A13occurrence.shape
compute_NCI(A13occurrence,14)
### 同じ結果なのがおかしい

[16.70149815252202, 11.684072248427416, 11.001149251665472, 10.572171972644648, 9.58724717687124, 7.830072004706212, 7.559729960931648, 6.945628950533069, 6.207138033130191, 5.70589573418924, 5.320699275302106, 4.0907268276269715, 4.000406083762532e-15]
1.9432890779738305


# トピックモデルの実装

In [150]:
from gensim.corpora import Dictionary
from gensim.models import word2vec
from gensim.models.ldamodel import LdaModel
import gensim

In [210]:
def get_corpus(keywords, max_document_frequency,min_document_frequency):
    dic = Dictionary(keywords)
    # max_document_frequency, min_document_frequency = 5, 2  # 単語の出現頻度の上限下限
    dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)  # 出現頻度
    corpus = [dic.doc2bow(key) for key in keywords]
    return(corpus)

In [217]:
####　関数内で定義されないので外で変数にしています。　対処したいです。
dic = Dictionary(keywords)
max_document_frequency, min_document_frequency = 5, 2  # 単語の出現頻度の上限下限
dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)

In [212]:
miscorpus=get_corpus(keywords,5,2)

##頻出ワードがカットできているかの確認

In [169]:
dic.token2id["する"]
# dic.id2token[1430]

KeyError: ignored

In [173]:
suru_cnt = 0
suru_doc_freq = 0
for bow in miscorpus:
  flag = False
  for id, cnt in bow:
    if id == dic.token2id["する"]:
      suru_cnt += cnt
      flag = True
  if flag:
    suru_doc_freq += 1
      
suru_cnt, suru_doc_freq

(962, 84)

##コーパスの作成

In [264]:
corpus =get_corpus(keywords,0.2,2)
####　関数内で定義されないので外で変数にしています。　対処したいです。
dic = Dictionary(keywords)
max_document_frequency, min_document_frequency = 0.2, 2  # 単語の出現頻度の上限下限
dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)

するが消えていることが分かる

In [265]:
# suru_cnt = 0
# suru_doc_freq = 0
# for bow in corpus:
#   flag = False
#   for id, cnt in bow:
#     if id == dic.token2id["する"]:
#       suru_cnt += cnt
#       flag = True
#   if flag:
#     suru_doc_freq += 1
      
# suru_cnt, suru_doc_freq

In [ ]:
corpus #IDと出現回数のbag of wordsになっている。

## LDAモデル化

In [266]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(corpus = corpus, id2word = dic, num_topics = 3, random_state = 1)

In [267]:
#結果
for t in lda.show_topics(num_words=30):
    print(t)

(0, '0.009*"." + 0.006*"６" + 0.005*"年度" + 0.005*"株" + 0.004*"池袋" + 0.004*"９" + 0.004*"取材" + 0.004*"８" + 0.004*"客" + 0.004*"会" + 0.004*"超える" + 0.003*"前" + 0.003*"事業" + 0.003*"証券" + 0.003*"経営" + 0.003*"店舗" + 0.003*"前年" + 0.003*"単価" + 0.003*"上場" + 0.003*"都" + 0.003*"高" + 0.003*"お金" + 0.003*"・" + 0.003*"製造" + 0.003*"株式" + 0.003*"営業" + 0.003*"さん" + 0.003*"銭" + 0.003*"増" + 0.003*"会社"')
(1, '0.007*"会社" + 0.004*"証券" + 0.004*"６" + 0.004*"ガソリン" + 0.004*"池袋" + 0.004*"９" + 0.004*"開業" + 0.004*"年度" + 0.004*"客" + 0.004*"・" + 0.004*"公開" + 0.003*"閉店" + 0.003*"補助" + 0.003*"店舗" + 0.003*"株" + 0.003*"費" + 0.003*"政府" + 0.003*"所" + 0.003*"上場" + 0.003*"関係" + 0.003*"家" + 0.003*"事業" + 0.003*"８" + 0.003*"営業" + 0.003*"投資" + 0.003*"新規" + 0.003*"全国" + 0.003*"取材" + 0.003*"店頭" + 0.003*"大手"')
(2, '0.006*"所" + 0.006*"工場" + 0.005*"値上げ" + 0.004*"政府" + 0.004*"ガソリン" + 0.004*"棒" + 0.004*"会社" + 0.004*"停止" + 0.004*"製油" + 0.004*"金" + 0.004*"和歌山" + 0.004*"会長" + 0.004*"生産" + 0.004*"抑制" + 0.003*"社" + 0.003*"高騰" + 0.003*"補助" + 0.0

In [270]:
num_words = 5
print("Topic words\n----------------------------------------------------")
# トピックごとの単語上位をnum_words個表示
for topic_num in range(lda.num_topics):
    word_weight_dict_by_topic = lda.show_topic(topic_num, num_words)  # トピックごとの単語と重みのリスト
    print(f"Topic {topic_num}:  {word_weight_dict_by_topic}")  

Topic words
----------------------------------------------------
Topic 0:  [('.', 0.009198594), ('６', 0.0058834776), ('年度', 0.0054593137), ('株', 0.0049415096), ('池袋', 0.0043707145)]
Topic 1:  [('会社', 0.007115846), ('証券', 0.004435431), ('６', 0.0042437008), ('ガソリン', 0.0042275195), ('池袋', 0.0039348044)]
Topic 2:  [('所', 0.006096076), ('工場', 0.005734924), ('値上げ', 0.005226761), ('政府', 0.004402456), ('ガソリン', 0.0043970463)]


文書が少ないのでほぼ同じになっている
topic2 などは　ガソリンの値上げが工場に影響している系だとわかる

##テストデータでやってみる

In [271]:
tests=["1月24日、ついにレギュラーガソリンの全国平均小売価格が170円を超えた。「長いコロナ不況で苦しいのに、ガソリン価格だけは右肩上がり。もう、ただのサラリーマンでは自家用車なんて持てないのかな」──そんな嘆きが、あちこちで聞かれるようになってきた。自動車業界は、燃料費の高騰だけが問題ではない。2年間も続いた未曾有の新型コロナウイルス禍。2022年こそは、ここまで控えられた分の“リベンジ消費”に期待がかかっていたが、蓋を開けてみると、買えるものも買えない状況に陥っている。都内のあるトヨタ系自動車販売店の営業マンは「コロナの世界的流行によって、半導体を生産する工場が、操業を何度も停止。部品供給が滞って、自動車工場もやむなく減産。去年8月に14年ぶりに全面改良したランドクルーザー（300系）も、問い合わせや注文は殺到しているのですが、納期は約4年後とお答えしているところなのです」と、肩を落としながら明かした。あるモータージャーナリストは「もともとランドクルーザーのセールスは、ほとんど海外。全生産の半分は中東、ロシアと豪州でも40％を占めて、日本の割合はわずか数％。供給台数が少ないので、さらに納期が遅れるのです」と補足した。さまざまな理由が重なって、新車の供給量で苦戦しているというのだ。また、高級車の代名詞でもあるメルセデス・ベンツの価格も上がってきている。かつて、400万円台から手を出すことのできたベンツCクラスは、「サラリーマンでも頑張ればベンツに乗れる象徴」として、多くのメルセデスユーザーを生んできた。しかし、昨年7年ぶりにフルモデルチェンジしたCクラスは、最低価格でも650万円以上。150万円以上の値上がりとなった。日本人の平均年収が約430万円（国税庁の「令和2年分　民間給与実態調査統計」より）だということを考えると、もはや「サラリーマンでは手が出せない超高級車」になってしまった。「しかも、そのベンツCクラスは半導体の不足により、当初予定していた装備が一部変更になっています。今後発売のロットは、高級スピーカーのブルメスターはオプションでも装着不可となり、トランクの下に足を入れると自動で開く『フットトランクオープナー』が付かないモデルもあります。しかもこれから新規発注すると納期は半年以上先とか……。AクラスやBクラスと比べて高級感があり、『ベビーSクラス』や『コンパクトSクラス』と言われていたCクラス。これだけ高額になったうえ、この仕様変更で購入を迷う人もいるかもしれませんね」（前出・モータージャーナリスト）もちろん、これらの変更はコロナ禍においては致し方ないこと。むしろ、半導体不足の中で、各自動車メーカーは新車を発売するために全力を注いでいるとは言える。一方で、車を購入予定の人の中には「新車が値上がりしたり納期に時間がかかる今、中古車はどうなの？」という気持ちになる人もいるかもしれない。だが、新車の供給不足の反動から、中古車市場は需要が高まり、値段も高騰している。",
      "大幅続伸。先週末に発表した自己株式の取得実施が買い材料視されている。発行済み株式数の4.23％に当たる300万株、10億円を取得上限としており、取得期間は2月21日から7月21日まで。機動的な資本政策の遂行及び株主への一層の利益還元を図ることを目的としている。当面の需給面での下支え材料につながるとの期待が先行へ。同社の自社株買いは、昨年10月にかけて上限10億円を実施して以来となる。"]

In [274]:
#どのトピックに分類されているか確認
def check_topic(test_texts,document,topic):
    test_keywords=get_keywords_list(test_texts)
    corpus_test = [dic.doc2bow(key) for key in test_keywords]
    pred = lda.inference(corpus_test)
    total=0
    for i in pred[0][document]:
        total += i
    return(pred[0][document][topic]/total)

In [276]:
check_topic(tests,0,2) #最初の文書のトピックは73%でtopic３

0.728828112674913

###備忘録

In [240]:
# test_keywords=get_keywords_list(tests)

In [ ]:
corpus_test = [dic.doc2bow(key) for key in test_keywords] #存在してなかったのは排除されている
# corpus_test = [dic.doc2bow(key, allow_update=True, return_missing=True) for key in keywords] #存在してなかったのを辞書に加えて更新

In [242]:
pred = lda.inference(corpus_test)
# print(pred) #各トピックのweightを表示している

(array([[  1.03,  39.31, 108.65],
       [ 23.32,   0.43,  12.25]], dtype=float32), None)


In [244]:
total=0
for i in pred[0][0]:
    total += i
pred[0][0][2]/total #全体のweigtの合計で割って％表示。　トピック3は73%とわかる

0.729201475299437

In [247]:
total=0
for i in pred[0][1]:
    total += i
pred[0][1][0]/total 

0.6477007929235954

#英語のfreesetを用いてやってみる

In [ ]:
https://newsdata.io/files/datasets/latest-news

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget -P /content/drive/MyDrive/SOCI https://newsdata.io/files/datasets/latest-news

--2022-02-21 13:25:36--  https://newsdata.io/files/datasets/latest-news
Resolving newsdata.io (newsdata.io)... 157.245.122.35
Connecting to newsdata.io (newsdata.io)|157.245.122.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159412320 (152M) [application/zip]
Saving to: ‘/content/drive/MyDrive/SOCI/latest-news’

latest-news         100%[===================>] 152.03M  11.4MB/s    in 14s     

2022-02-21 13:25:51 (10.7 MB/s) - ‘/content/drive/MyDrive/SOCI/latest-news’ saved [159412320/159412320]



In [7]:
!cp /content/drive/MyDrive/SOCI/latest-news .
!unzip latest-news

Archive:  latest-news
replace Latest_News/Latest_News.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

NameError: ignored

In [12]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Profmike/Latest_News.csv")
df["description"]

0        La Gazzetta dello Sport racconta del nervosism...
1        Il tecnico del Napoli è stato sanzionato per l...
2        Gli ultras del Napoli potrebbero rientrare all...
3        Decisivo nella costruzione del gioco e nella f...
4        Dopo il negativo inizio di stagione il preside...
                               ...                        
86555    Duminică, 24 octombrie, este o zi de sărbătoar...
86556    The group described Mr Tinubu as an exceptiona...
86557    Jakub Świerczok strzelił bramkę w meczu 33. ko...
86558    A woman in her 70s has died following a house ...
86559    “문재인 정부 성공과 정권 재창출 위해서 작은 힘이나마 보태겠습니다.”(이낙연 전 ...
Name: description, Length: 86560, dtype: object

In [11]:
df

,title,link,keywords,creator,video_url,description,content,pubDate,full_description,image_url,source_id
0,Spalletti arrabbiato con i suoi calciatori nei...,https://www.areanapoli.it/rassegna-stampa/spal...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,La Gazzetta dello Sport racconta del nervosism...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
1,Squalifica e multa per Spalletti: la reazione ...,https://www.areanapoli.it/rassegna-stampa/squa...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,Il tecnico del Napoli è stato sanzionato per l...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
2,"Napoli-Bologna, gli ultras potrebbero entrare ...",https://www.areanapoli.it/rassegna-stampa/napo...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,Gli ultras del Napoli potrebbero rientrare all...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
3,"Juve, Dybala vale doppio: ecco perch&eacute; p...",https://www.tuttosport.com/news/calcio/serie-a...,['Juventus'],NaN,NaN,Decisivo nella costruzione del gioco e nella f...,NaN,2021-10-26 07:05:03,NaN,https://cdn.tuttosport.com/img/600/400/2021/10...,tuttosport
4,Tubertini non è più l'allenatore di Siena,https://www.tuttosport.com/news/pallavolo/a2-m...,"['Tubertini', 'Emma Villas']",NaN,NaN,Dopo il negativo inizio di stagione il preside...,NaN,2021-10-26 07:05:03,NaN,https://cdn.tuttosport.com/img/600/400/2021/10...,tuttosport
...,...,...,...,...,...,...,...,...,...,...,...
86555,Ziua cea mare a derbiurilor: Regal fotbalistic...,https://sport.hotnews.ro/stiri-fotbal-25130203...,['Sport/Fotbal'],NaN,NaN,"Duminică, 24 octombrie, este o zi de sărbătoar...",NaN,2021-10-24 08:51:00,"Duminică, 24 octombrie, este o zi de sărbătoar...",NaN,hotnews
86556,2023: Group mobilises support for Tinubu,https://www.premiumtimesng.com/news/more-news/...,"['More News', '2023 presidential election', 'A...",['Agency Report'],NaN,The group described Mr Tinubu as an exceptiona...,NaN,2021-10-24 08:50:59,"Umar Mohammed, Director-General, Tinubu Suppor...",NaN,premiumtimesng
86557,"Liga japońska. Jakub Świerczok z golem, remis ...",https://sport.interia.pl/pilka-nozna/news-liga...,['Piłka nożna'],NaN,NaN,Jakub Świerczok strzelił bramkę w meczu 33. ko...,NaN,2021-10-24 08:50:57,"Rozgrywki J1 League, najwyższego poziomu klubo...",https://i.iplsc.com/liga-japonska-jakub-swierc...,interiasport
86558,Woman (70s) dies following house fire in Cork ...,https://www.irishtimes.com/news/ireland/irish-...,NaN,NaN,NaN,A woman in her 70s has died following a house ...,NaN,2021-10-24 08:50:53,A woman in her 70s has died following a house ...,https://www.irishtimes.com/polopoly_fs/1.47092...,irishtimes
